In [3]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import glob
from torchvision import transforms

In [4]:
df = pd.read_excel('all_patients.xlsx')
df = df[df.shape_VoxelVolume > 125]
df.drop(df[df['labels'] == 3].index, inplace = True) ## dropping label of the call OTHERS
df.drop(df[df['number_annotations'] > 1].index, inplace = True)
df

,subjid,label,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,...,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength,Age,number_annotations,F,M,labels
0,Oslo01,1,0.829356,0.682981,8.512717,12.464063,14.866069,14.560220,12.529964,15.394804,...,158.977235,0.008929,0.302273,0.074659,0.008950,58,1,0,1,0
1,Oslo01,2,0.944143,0.900681,14.133927,15.692487,17.804494,19.416488,18.681542,19.467922,...,39.445794,0.007407,0.115448,0.010426,0.007130,58,1,0,1,0
4,Oslo02,4,0.849295,0.841082,8.703783,10.348315,10.816654,11.704700,11.704700,12.449900,...,14.239428,0.022868,0.159307,0.020316,0.023465,50,1,1,0,1
5,Oslo02,5,0.939828,0.821847,12.093804,14.715396,17.262677,17.888544,17.492856,18.000000,...,49.609354,0.008536,0.137901,0.028455,0.008212,50,1,1,0,1
7,Oslo03,1,0.930698,0.629604,10.914986,17.336268,20.615528,18.439089,19.416488,20.712315,...,43.648474,0.007781,0.123220,0.017415,0.007982,64,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,Stan_328,1,0.810571,0.325076,8.769420,26.976501,31.384710,28.160256,27.459060,33.075671,...,12.988427,0.019891,0.030104,0.000639,0.019768,53,1,0,1,1
2415,Stan_328,8,0.967496,0.597282,6.131668,10.265944,11.661904,12.369317,12.165525,12.688578,...,18.452502,0.024791,0.208465,0.046975,0.023205,53,1,0,1,1
2418,Stan_328,11,0.561746,0.506390,5.004740,9.883181,8.246211,11.704700,10.000000,12.124356,...,0.553323,0.468447,0.025263,0.000748,0.365681,53,1,0,1,1
2419,Stan_328,12,0.959775,0.756903,5.164755,6.823538,7.615773,8.602325,8.602325,8.602325,...,0.000000,1000000.000000,0.000000,0.000000,0.000000,53,1,0,1,1


In [5]:
new_df = pd.DataFrame({'Subjid': df.iloc[:, 0], 'Tumor': df.iloc[:, 1], 'Labels': df.iloc[:, -1]})
new_df = new_df.drop_duplicates(subset='Subjid', keep='first')
#new_df.to_excel('labels_DL.xlsx', index = False)
new_df

,Subjid,Tumor,Labels
0,Oslo01,1,0
4,Oslo02,4,1
7,Oslo03,1,0
11,Oslo04,1,0
13,Oslo06,1,1
...,...,...,...
2359,Stan_318,1,1
2369,Stan_319,2,1
2387,Stan_323,3,1
2408,Stan_328,1,1


In [6]:
stan = new_df['Subjid'][59:].to_list()
oslo = new_df['Subjid'][:59].to_list()

## we have created a new folder called deep learning with all the folder with all the folder per patient labels as 0, 1 or 2 with tumors greater than 125mm3

In [324]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    new_folder_name = folder_name.replace("Subject", "Oslo")
    #new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [325]:
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))

for patient_folder in patient_folders:
    #id = patient_folder[-2:]
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    #print(seg_folders)
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            #print(seg_folder)
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            #print(image_files)
            # Delete all the image files except for 'seg_1'
            for file in image_files:
            # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)

In [326]:
patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-6:]
    if id not in oslo:
        shutil.rmtree(patient_folder)

In [327]:
import os

directory = '/data/projects/TMOR/data/Deeplearning/'

patient_folders = glob.glob(os.path.join(directory, 'Oslo*'))


# Iterate over patient folders
for patient_folder in patient_folders:
    #patient_path = os.path.join(directory, patient_folder)
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    #print(patient_path)
    # Find segmentation files in the patient folder
    #seg_files = [f for f in os.listdir(patient_path) if f.startswith('seg_')]
    #seg_folders = glob.glob(os.path.join(patient_path, 'seg*'))
    #print(patient_folder)
    # Rename segmentation files
    for seg_folder in seg_folders:
        seg_old_path = seg_folder
        seg_new_path = os.path.join(patient_folder, 'seg.nii.gz')
        os.rename(seg_old_path, seg_new_path)
        #print(seg_new_path)

## STANDFORD DATA

In [328]:
# Specify the path to the parent folder
parent_folder = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all subfolders within the parent folder
subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

# Iterate over the subfolders and rename them

for folder in subfolders:
    folder_name = os.path.basename(folder)
    #new_folder_name = folder_name.replace("Subject", "Oslo")
    new_folder_name = folder_name.replace("Mets", "Stan")
    new_folder_path = os.path.join(parent_folder, new_folder_name)
    os.rename(folder, new_folder_path)


In [329]:
import os
import glob
#delete the other file for OSLO
directory = '/data/projects/TMOR/data/Deeplearning/'

# Get a list of all patient folders
patient_folders = glob.glob(os.path.join(directory, 'Stan*'))

for patient_folder in patient_folders:
    #id = patient_folder[-2:]
    seg_folders = glob.glob(os.path.join(patient_folder, 'seg*'))
    t1_gds = glob.glob(os.path.join(patient_folder, 't1_gd*'))
    #print(seg_folders)
    # Iterate over each seg folder
    for seg_folder in seg_folders:
        for t1_gd in t1_gds:
            #print(seg_folder)
            image_files = glob.glob(os.path.join(patient_folder, '*'))
            #print(image_files)
            # Delete all the image files except for 'seg_1'
            for file in image_files:
                # Check if the file is in the segmentation folder or T1-Gd folder
                if file not in [seg_folder, t1_gd]:
                    os.remove(file)


In [330]:
#Remove all the files that are not within stan
import shutil
for patient_folder in patient_folders:
    id = patient_folder[-8:]
    if id not in stan:
        shutil.rmtree(patient_folder)

In [333]:

class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir= img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform
        #self.data_files = [os.path.basename(file) for file in self.img]


    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        #data_file = self.data_files[index]
        seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
        label = self.img_labels.iloc[idx, 2]

        # Load data and mask using nibabel
        data = nib.load(seg_path).get_fdata()
        mask = nib.load(img_path).get_fdata()

        # Load label for the corresponding data file
        #label = self.labels[data_file]

        # Apply transformations (if any)
        if self.transform is not None:
            data = self.transform(data)
            mask = self.transform(mask)

        # Normalize data and convert to torch tensors
        data = torch.from_numpy(data).unsqueeze(0).float()
        mask = torch.from_numpy(mask).unsqueeze(0).float()
        label = torch.tensor(label).long()

        return data, mask, label


In [334]:
img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir,labels_file, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


In [335]:
print("Dataset length:", len(dataset))

# Print information for a specific sample
index = 0  # Index of the sample
sample = dataset[index]
images, label = sample
print("Sample images:", images)
print("Label:", label)

Dataset length: 438


TypeError: Unexpected type <class 'numpy.ndarray'>

In [336]:
# Create an instance of the dataset
dataset = BrainSegmentationDataset(img_dir=img_dir, labels_file=labels_file)

print("Dataset length:", len(dataset))

# Access an item from the dataset
sample_data, sample_mask, sample_label = dataset[118]

# Verify the shapes of the data, mask, and label
print('Data shape:', sample_data.shape)
print('Mask shape:', sample_mask.shape)
print('Label:', sample_label)


Dataset length: 438
Data shape: torch.Size([1, 127, 172, 136])
Mask shape: torch.Size([1, 127, 172, 136])
Label: tensor(1)


## lets try divide by train an test

In [4]:
import os
import torch
from torch.utils.data import Dataset
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split

class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        if self.transform is None:
            seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]
            data = nib.load(seg_path).get_fdata()
            mask = nib.load(img_path).get_fdata()

        else:
            seg_path = os.path.join(self.img_dinr, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]

            data = nib.load(seg_path).get_fdata()
            mask = nib.load(img_path).get_fdata()

        data = torch.from_numpy(data).unsqueeze(0)
        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return data, mask, label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y ,random_state=0)

        X_train = self.process_images(X_train)
        X_test = self.process_images(X_test)

        return X_train, X_test, y_train, y_test

    def process_images(self, image_names):
        image_tensors = []

        for image_name in image_names:
            if self.transform is None:
                seg_path = os.path.join(self.img_dir, self.img_labels.iloc[image_name, 0], 'seg.nii.gz')
                img_path = os.path.join(self.img_dir, self.img_labels.iloc[image_name, 0], 't1_gd.nii.gz')

                data = nib.load(seg_path).get_fdata()
                mask = nib.load(img_path).get_fdata()

            else:
                data = self.transform(data)
                mask = self.transform(mask)

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors


In [7]:
img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
#transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir,labels_file, transform=None)
#data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

ValueError: Location based indexing can only have [integer, integer slice (START point is INCLUDED, END point is EXCLUDED), listlike of integers, boolean array] types

In [315]:
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test

In [316]:
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test
X_train[0][1].sum(axis=0).sum(axis=1).unique()

tensor([-113.0706, -112.6847, -109.6506,  ...,  111.3742,  112.9775,
         114.3297], dtype=torch.float64)

In [312]:
y_train

array([1, 1, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 0, 2, 0, 1, 0, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1,
       1, 2, 0, 0, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1,
       1, 0, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2,
       1, 1, 1, 1, 2, 0, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 2, 1, 1, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 2, 1, 1, 1, 1, 1, 1, 0, 2, 1,
       1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,

In [ ]:
import os
import torch
from torch.utils.data import Dataset
import nibabel as nib
import pandas as pd
from sklearn.model_selection import train_test_split

class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        seg_path = os.path.join(self.img_dir, self.X_train.iloc[idx, 0], 'seg.nii.gz')
        img_path = os.path.join(self.img_dir, self.X_train.iloc[idx, 0], 't1_gd.nii.gz')
        label = self.y_train[idx]

        data = self.preprocess_image(seg_path)
        mask = self.preprocess_image(img_path)

        if self.transform is not None:
            data = self.transform(data)
            mask = self.transform(mask)

        data = torch.from_numpy(data).unsqueeze(0).float()
        mask = torch.from_numpy(mask).unsqueeze(0).float()
        label = torch.tensor(label).long()

        return data, mask, label

    def preprocess_image(self, image_path):
        image = nib.load(image_path).get_fdata()
        # Perform preprocessing steps here (e.g., normalization, resizing, cropping, etc.)
        # Preprocess image and return the processed image

        return processed_image

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        X_train = pd.DataFrame(X_train, columns=['Filename'])
        X_test = pd.DataFrame(X_test, columns=['Filename'])

        return X_train, X_test, y_train, y_test


In [18]:
class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        if self.transform is None:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            data_1 = nib.load(img_path)
            data_2 = self.convert_to_grayscale(data_1)
            data = data_2.get_fdata()
            mask_1 = nib.load(seg_path)
            mask_2 = self.convert_to_grayscale(mask_1)
            mask = mask_2.get_fdata()
            label = self.y_train[idx]
        else:
            # Apply the transformation to the mask
            seg_path = os.path.join(self.img_dinr, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            data_1 = nib.load(img_path)
            data_2 = self.transform(data_1)
            data_3 = self.convert_to_grayscale(data_2)
            data = data_3.get_fdata()
            mask_1 = nib.load(seg_path)
            mask_2 = self.transform(mask_1)
            mask_3 = self.convert_to_grayscale(mask_2)
            mask = mask_3.get_fdata()
            label = self.y_train[idx]


        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return mask, data,label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

        X_train = self.process_images(X_train)
        X_test = self.process_images(X_test)

        return X_train, X_test, y_train, y_test

    def process_images(self, image_names):
        image_tensors = []
        #pene = list(image_names[:, 0])
        for image_name in image_names:
            if self.transform is None:
                # Load the mask directly
                seg_path = os.path.join(self.img_dir, image_name, 'seg.nii.gz')
                img_path = os.path.join(self.img_dir, image_name, 't1_gd.nii.gz')
                data_1 = nib.load(img_path)
                data_2 = self.convert_to_grayscale(data_1)
                data = data_2.get_fdata()
                mask_1 = nib.load(seg_path)
                mask_2 = self.convert_to_grayscale(mask_1)
                mask = mask_2.get_fdata()

            else:
                # Apply the transformation to the mask
                seg_path = os.path.join(self.img_dir, image_name, 'seg.nii.gz')
                img_path = os.path.join(self.img_dir, image_name, 't1_gd.nii.gz')
                data_1 = nib.load(img_path)
                data_2 = self.transform(data_1)
                data_3 = self.convert_to_grayscale(data_2)
                data = data_3.get_fdata()
                mask_1 = nib.load(seg_path)
                mask_2 = self.transform(mask_1)
                mask_3 = self.convert_to_grayscale(mask_2)
                mask = mask_3.get_fdata()


            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors


img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir, labels_file, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

TypeError: Unexpected type <class 'nibabel.nifti1.Nifti1Image'>

In [34]:
def crop_to_largest_tumor(tensor, crop_size):
    # Get tumor dimensions
    tumor_size = np.sum(tensor > 0, axis=(1, 2, 3))
    max_tumor_size = np.max(tumor_size)

    # Calculate cropping indices
    start = (tumor_size - max_tumor_size) // 2
    end = start + max_tumor_size

    # Perform cropping
    cropped_tensor = tensor[:, start:end, start:end, start:end]

    return cropped_tensor
class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        # Load the mask directly
        seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
        data_1 = nib.load(img_path)
        data = data_1.get_fdata()
        mask_1 = nib.load(seg_path)
        mask = mask_1.get_fdata()
        label = self.y_train[idx]

        data = torch.from_numpy(data).unsqueeze(0)
        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return mask, data, label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

        X_train = self.process_images(X_train)
        X_test = self.process_images(X_test)

        return X_train, X_test, y_train, y_test

    def process_images(self, image_names):
        image_tensors = []

        for image_name in image_names:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, image_name, 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, image_name, 't1_gd.nii.gz')
            data_1 = nib.load(img_path)
            data = data_1.get_fdata()
            mask_1 = nib.load(seg_path)
            mask = mask_1.get_fdata()

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors

            tumor_size = np.sum(mask > 0, axis=(0, 1, 2))
            print(tumor_size)
            max_tumor_size = np.max(tumor_size)
            print(max_tumor_size)

            # Calculate cropping indices
            start = (tumor_size - max_tumor_size) // 2
            print(start)
            end = start + max_tumor_size

            # Crop image and mask tensors
            data = data[start[0]:end[0], start[1]:end[1], start[2]:end[2]]
            mask = mask[start[0]:end[0], start[1]:end[1], start[2]:end[2]]

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors

img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir, labels_file, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


1344
1344
0


IndexError: invalid index to scalar variable.

In [ ]:
def preprocess_nifti_scan(file_path, threshold_value=0.5):
    # Load the NIfTI scan using nibabel
    nifti_img = nib.load(file_path)

    # Get the scan data and header
    scan_data = np.array(nifti_img.dataobj)
    scan_header = nifti_img.header

    # Apply preprocessing steps
    # Replace this section with your specific preprocessing operations

    # Normalize the scan data
    scan_data = (scan_data - np.mean(scan_data)) / np.std(scan_data)

    # Convert the scan data to the desired format
    # (e.g., float32, 0-1 range, etc.)
    scan_data = scan_data.astype(np.float32)
    scan_data = np.clip(scan_data, 0.0, 1.0)

    # Get the tumor bounding boxes
    tumor_bounding_boxes = get_tumor_bounding_boxes(scan_data, threshold_value)

    # Generate the labels for each bounding box
    labels = [1] * len(tumor_bounding_boxes)

    # Return the preprocessed data and labels
    return scan_data, labels


def get_tumor_bounding_boxes(scan_data, threshold_value=0.5):
    # Threshold the scan data to separate tumors from background
    thresholded_scan = np.where(scan_data >= threshold_value, 1, 0)

    # Perform connected component analysis to identify separate tumor regions
    num_labels, labeled_scan = cv2.connectedComponents(thresholded_scan.astype(np.uint8))

    # Iterate through each label (excluding the background label) and extract bounding boxes
    tumor_bounding_boxes = []
    for label in range(1, num_labels):
        # Extract the region corresponding to the current label
        region_mask = np.where(labeled_scan == label, 1, 0)

        # Find the bounding box coordinates using contour extraction
        contours, _ = cv2.findContours(region_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        x, y, w, h = cv2.boundingRect(contours[0])

        # Append the bounding box coordinates to the list
        tumor_bounding_boxes.append((x, y, w, h))

    return tumor_bounding_boxes


class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None, threshold_value=0.5):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform
        self.threshold_value = threshold_value

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        # Load the mask directly
        seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
        data_1 = nib.load(img_path)
        data = data_1.get_fdata()
        mask_1 = nib.load(seg_path)
        mask = mask_1.get_fdata()
        label = self.y_train[idx]

        data = torch.from_numpy(data).unsqueeze(0)
        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        # Preprocess the data and mask
        data, mask = preprocess_nifti_scan(img_path, self.threshold_value)

        if self.transform:
            data = self.transform(data)

        if self.target_transform:
            mask = self.target_transform(mask)

        return mask, data, label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

        X_train = self.process_images(X_train)
        X_test = self.process_images(X_test)

        return X_train, X_test, y_train, y_test

    def process_images(self, image_names):
        image_tensors = []

        for image_name in image_names:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, image_name, 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, image_name, 't1_gd.nii.gz')
            data_1 = nib.load(img_path)
            data = data_1.get_fdata()
            mask_1 = nib.load(seg_path)
            mask = mask_1.get_fdata()

            data_tensor = torch.from_numpy(data).unsqueeze(0)
            mask_tensor = torch.from_numpy(mask).unsqueeze(0)

            image_tensors.append((data_tensor, mask_tensor))

        return image_tensors


In [29]:
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test
X_train[0][1].sum(0).sum(1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [20]:
pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 41.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [119]:
seg_dir = '/data/projects/TMOR/data/Deeplearning/'
for subj_id in os.listdir(seg_dir):
    #extrasct the correct id from the subjects
    #id = int(subj_id[7:9])
    # Define paths to the segmentation and regular t1 postprocessed files for this subject
    seg_glob = os.path.join(seg_dir, f'{subj_id}', f'seg.nii.gz')
    reg_glob = os.path.join(seg_dir, f'{subj_id}', f't1_gd.nii.gz')
    seg_paths = glob.glob(seg_glob)
    reg_paths = glob.glob(reg_glob)
    # Loop through all matching segmentation and registration files
    for seg, reg in zip(seg_paths, reg_paths):
        if os.path.exists(seg) and os.path.exists(reg):
            print(seg)
            mask = nib.load(seg)
            vol_mask = mask.get_fdata()
            klk = 0
            for label in np.unique(vol_mask):
                indices = np.argwhere(vol_mask == label)
                print(klk, len(indices))
                klk += 1
                #print(indices)

    break # Iterate over the indices
            #for index in indices:
                #row_idx, col_idx, slice_idx = index

                # Access the index of seg_paths
                #seg_index = seg_paths.index(seg)

                # Do something with the index and indices
                #print(f"Index: {seg_index}, Coordinate: ({row_idx}, {col_idx}, {slice_idx})")
                #break

/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
0 3190750
1 47
2 553
3 1710
4 26


In [111]:
for label in np.unique(vol_mask):
    print(label)

0.0
1.0
2.0
3.0
4.0


In [118]:
pene

5

In [71]:
file_path = "/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz"
nifti_img = nib.load(file_path)
scan_data = np.array(nifti_img.dataobj)
#scan_data = nifti_img.get_fdata()
threshold_value = 0.5

# Call the get_tumor_bounding_boxes function
tumor_bounding_boxes = get_tumor_bounding_boxes(scan_data, threshold_value)

# Print the obtained tumor bounding boxes
print(tumor_bounding_boxes)

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/connectedcomponents.cpp:5623: error: (-215:Assertion failed) L.channels() == 1 && I.channels() == 1 in function 'connectedComponents_sub1'


In [62]:
file_path = "/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz"
nifti_img = nib.load(file_path)
scan_data = np.array(nifti_img.dataobj)
thresholded_scan = np.where(scan_data >= 0.5, 1, 0)
thresholded_scan = thresholded_scan.astype(np.uint8)
np.unique(thresholded_scan)

array([0, 1], dtype=uint8)

In [27]:
thresholded_scan, num_labels, tumor_bounding_boxes = get_tumor_bounding_boxes(scan_data, threshold_value)

print(thresholded_scan)

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/connectedcomponents.cpp:5623: error: (-215:Assertion failed) L.channels() == 1 && I.channels() == 1 in function 'connectedComponents_sub1'


In [440]:
class BrainSegmentationDataset(Dataset):
    def __init__(self, img_dir, labels_file, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_labels = pd.read_excel(labels_file)
        self.transform = transform
        self.target_transform = target_transform

        self.X_train, self.X_test, self.y_train, self.y_test = self.split_data()

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        if self.transform is None:
            # Load the mask directly
            seg_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]
            data = nib.load(img_path).get_fdata()
            mask = nib.load(seg_path).get_fdata()
        else:
            # Apply the transformation to the mask
            seg_path = os.path.join(self.img_dinr, self.img_labels.iloc[idx, 0], 'seg.nii.gz')
            img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0], 't1_gd.nii.gz')
            label = self.y_train[idx]
            mask = nib.load(seg_path).get_fdata()
            data = nib.load(img_path).get_fdata()

        mask = torch.from_numpy(mask).unsqueeze(0)
        label = torch.tensor(label).long()

        return mask, data,label

    def split_data(self):
        X = self.img_labels.iloc[:, 0].values
        y = self.img_labels.iloc[:, 2].values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)


        return X_train, X_test, y_train, y_test


img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
dataset = BrainSegmentationDataset(img_dir, labels_file)

# Access the X_train list
print(dataset.X_train.size)

350


In [444]:
img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = 'labels_DL.xlsx'
transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])

dataset = BrainSegmentationDataset(img_dir,labels_file, transform=None)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
X_train

In [348]:
X_train, X_test, y_train, y_test = dataset.X_train, dataset.X_test, dataset.y_train, dataset.y_test

X_train[6].shape

torch.Size([1, 135, 159, 132])

In [403]:
# Assuming you have a DataFrame called "data"
# Features are stored in columns X1, X2, X3, ...
# The target variable is stored in column 'y'

X = new_df.drop(['Labels', 'Tumor'],axis=1)  # Features
y = new_df['Labels']  # Target variable
# Random split into 80% train and 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

img_dir = '/data/projects/TMOR/data/Deeplearning/'
labels_file = pd.read_excel('labels_DL.xlsx')

def process_images(img_dir, labels_file):
    image_tensors = []
    pene = list(labels_file.iloc[:, 0])
    for image_name in pene:
        seg_path = os.path.join(img_dir, image_name, 'seg.nii.gz')
        print(seg_path)
        img_path = os.path.join(img_dir, image_name, 't1_gd.nii.gz')
        mask = nib.load(seg_path).get_fdata()
        data = nib.load(img_path).get_fdata()

    data_tensor = torch.from_numpy(data).unsqueeze(0)
    mask_tensor = torch.from_numpy(mask).unsqueeze(0)

    image_tensors.append((data_tensor, mask_tensor))

    return image_tensors

processed_images = process_images(img_dir, labels_file)

/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo01/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo02/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo03/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo04/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo06/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo07/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo08/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo09/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo10/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo11/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo12/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo13/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo14/seg.nii.gz
/data/projects/TMOR/data/Deeplearning/Oslo17/seg

KeyboardInterrupt: 

In [431]:
pene = list(X_train.iloc[:, 0])

for i in pene:
    print(i)

Stan_265
Oslo62
Stan_197
Stan_058
Stan_319
Oslo47
Stan_026
Stan_165
Oslo31
Stan_260
Stan_049
Stan_312
Oslo41
Stan_045
Oslo26
Stan_251
Stan_285
Oslo13
Stan_051
Stan_028
Stan_183
Oslo32
Stan_064
Stan_203
Oslo46
Oslo06
Oslo37
Stan_009
Stan_315
Stan_024
Oslo20
Stan_184
Oslo12
Stan_126
Stan_120
Oslo01
Stan_144
Stan_019
Oslo49
Stan_054
Oslo33
Oslo45
Stan_291
Oslo30
Oslo28
Stan_134
Oslo44
Stan_059
Stan_102
Stan_318
Stan_246
Oslo52
Stan_148
Stan_010
Stan_038
Oslo38
Stan_121
Stan_136
Stan_171
Stan_014
Stan_087
Stan_098
Oslo60
Oslo07
Stan_132
Stan_170
Oslo54
Oslo40
Stan_052
Stan_047
Oslo39
Oslo51
Oslo09
Oslo48
Stan_033
Stan_307
Stan_072
Stan_055
Stan_289
Stan_290
Stan_068
Oslo10
Stan_142
Oslo17
Stan_005
Stan_257
Oslo04
Oslo21
Oslo43
Stan_037
Stan_266
Oslo08
Stan_244
Stan_101
Oslo03
Stan_173
Stan_016
Oslo61
Stan_123
Oslo55
Stan_323
Oslo66
Oslo53
Stan_066
Oslo25
Oslo65
Stan_316
Stan_227
Oslo42
Stan_311
Oslo02
Oslo59
Stan_230
Stan_111
Stan_100
Stan_149
Stan_065
Stan_039
Stan_176
Stan_338
Oslo24
Sta

In [419]:
X_train.iloc[4, :]

Subjid    Stan_319
Name: 2369, dtype: object